#### Análise de perfil de produtos com Python

A presente análise traz um perfil detalhado sobre os produtos da companhia ao longo de agosto de 2014 a abril de 2018. Serão geradas informações relativas aos produtos pertencentes a cada departamento, às quantidades vendidas, aos produtos que mais vendem, entre outros. Essas informações foram extraídas da base de dados resultante da combinação entre tabelas Fato: *Detalhes* e *Cabecalho* e Dimensão: *Categoria*, *Funcionario* e *Escritorio*. Os resultados obtidos a partir dessa análise podem contribuir para que sejam adotadas melhores práticas que visem aumentar o desempenho da empresa.

##### Bibliotecas

In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
from anytree import Node, RenderTree

##### Relacionamento entre as bases *fato* e *dimensões*

In [2]:
#Importando a tabela fato união entre Cabecalho e Detalhes

fatos = pd.read_csv("fatos.csv", decimal = ",")

In [3]:
#Importando as tabelas dimensões

#Dimensão Categoria
d_cat = pd.read_excel("Dimensoes_DadosModelagem.xlsx", sheet_name=3)
#Dimensão Produto
d_prod = pd.read_excel("Dimensoes_DadosModelagem.xlsx", sheet_name=4)
#Dimensão Funcionários
d_func = pd.read_excel("Dimensoes_DadosModelagem.xlsx", sheet_name= 1)
#Dimensão Escritórios
d_esc = pd.read_excel("Dimensoes_DadosModelagem.xlsx", sheet_name= 2)


In [4]:
#Relacionamento: entre d_prod e d_cat
data = pd.merge(left = d_prod, right = d_cat, on = ["CategoriaID"], how = "inner")

#Relacionamento: entre data e f_cab
data = pd.merge(left = data, right = fatos, on = ["ProdutoID"], how = "inner")

#Relacionamento: entre data e d_func
data = pd.merge(left = data, right = d_func,  on = ["FuncionarioID"], how = "inner")

#Relacionamento: entre data e d_esc
data = pd.merge(left = data, right = d_esc,  left_on = ["Escritorio"], right_on = ["Escritorios"], how = "inner")

##### Descrição do período analisado

Data inicial e final da análise:

In [5]:
#Convertendo a coluna Data em data
data['Data']= pd.to_datetime(data['Data'], format = "%d/%m/%Y")

print("Data inicial:", min(data['Data']))

print("Data final:", max(data['Data']))

Data inicial: 2014-07-30 00:00:00
Data final: 2018-04-07 00:00:00


Criando variáveis para ano e meses:

In [6]:
#Criando a variável ano
data['Ano'] = pd.DatetimeIndex(data['Data']).year

#Criando a variável mês
data['Mês'] = pd.DatetimeIndex(data['Data']).month

Descobrindo quais os meses de venda para cada ano:

In [7]:
data.groupby("Ano")['Mês'].apply(lambda x: list(np.unique(x)))

Ano
2014                      [7, 8, 9, 10, 11, 12]
2015    [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
2016    [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
2017    [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
2018                               [1, 2, 3, 4]
Name: Mês, dtype: object

É importante destacar aqui que os dados de 2014 e 2018 estão incompletos. Assim possivelmente seus resultados financeiros relativos a produtos serão menores. 

##### Hierarquia para identificar departamentos e produtos

In [8]:
#Selecioando as coluanas referentes a departamento e produto
df = data.copy()
df = df[['Departamento', 'Produto']]

#Transformando dataframe em lista
df = df.values.tolist()


In [9]:
#Definindo uma função que destaca a segmentação entre departamento e produtos

def add_nodes(nodes, parent, child):
    if parent not in nodes:
        nodes[parent] = Node(parent)  
    if child not in nodes:
        nodes[child] = Node(child)
    nodes[child].parent = nodes[parent]


df = pd.DataFrame(columns=["Parent","Child"], data= df)
nodes = {}  
for parent, child in zip(df["Parent"],df["Child"]):
    add_nodes(nodes, parent, child)

roots = list(df[~df["Parent"].isin(df["Child"])]["Parent"].unique())
for root in roots:  
    for pre, _, node in RenderTree(nodes[root]):
        print("%s%s" % (pre, node.name))


Clothing
├── Lenin Jeansshorts
├── Mr X Trousers
├── O-Man Underwear
├── Desperado Jeans
├── Samba Socker  Socks
├── US-Master Jeans
├── Atles Lussekofta
├── Bow tie
├── Chantell Shirt
├── Oyaki Kimono
├── Shagall Socks
├── Tuxedo
├── Sumi Underwear
├── Le Baby Dress
├── Terence Top
├── Okkaba Skin Jackets
├── Jumpin Jack Flash Dress
├── Serve-Shirt
├── Stretch oui-pants
├── Skirt
├── Conserve Shoes
├── RDL Suit
├── Nikee Running Shoes
├── Sapporoo Gloves
├── Aino Shoes
├── Mehmet-Tröja
├── Duck Trousers
├── Duck Shirt
├── Rodbye Troje
├── Mehmet-Napp
├── Rossi Bermuda Shorts
├── Rossi Bikkini
├── Baywatch Bikkini
├── LA. Shorts
├── Summer Shorts
├── Rossi Shorts
├── Wimbledon T-Shirt
├── Squash Shorts
├── Tennis Suit
├── Basket Shoes
├── Basket Vest
├── Bike Helmet
├── Car Boots
├── Finnish Sport Blades
├── Finnish Swimsuit
├── Patamonia Fleece Jacket
├── Feiss Fleece Trousers
├── Rasta WCT
├── Game Over T-Shirt
├── Adihash Running Shoe
├── Sheat Shoes
├── Cap
├── Racing Truck  Socks


#### Produto e quantidade vendida

##### Quantidade de produtos por departamento

In [10]:
produtos = data.groupby(['Departamento']).Produto.agg(Quantidade = 'nunique')
produtos.loc['Total'] = produtos.sum()
produtos

,Quantidade
Departamento,
Clothing,63
Shoes,14
Total,77


O portfólio da companhia analisada contém 77 produtos, sendo 63 correspondentes ao departamento **Clothing** e 14 ao departamento **Shoes**.

##### Quantidade de produtos vendidos por ano e departmento

In [11]:
produtos = data.groupby(['Ano','Departamento']).Quantidade.agg(Quantidade = 'sum')
produtos.loc['Total'] = produtos.sum()
produtos

,Quantidade
"(2014, Clothing)",782
"(2014, Shoes)",54
"(2015, Clothing)",7142
"(2015, Shoes)",1328
"(2016, Clothing)",12149
"(2016, Shoes)",2374
"(2017, Clothing)",20118
"(2017, Shoes)",4231
"(2018, Clothing)",3384
"(2018, Shoes)",495


Diante aos resultados observa-se que o departamento **Clothing** além de ter o maior número de produtos dentro da empresa também é responsável pela maior quantidade vendida de produtos em todos os anos. Destaque-se o ano de 2017 que obteve o melhor resultado de quantidade vendida para os dois departamentos.  

##### Os 5 produtos mais vendidos por ano e suas respectivas quantidades

In [12]:
data.groupby(['Ano', 'Produto']).Quantidade.agg(Qtde = 'sum').sort_values(['Ano','Qtde'],  ascending=False).groupby(level=0).head(5)

Qtde
Ano  Produto                    
2018 Bow tie                 234
     Feiss Fleece Trousers   195
     Rasta WCT               185
     Squash Shorts           160
     Ga-Ga Dress             150
2017 Game Over T-Shirt       863
     Samba Socker  Socks     764
     Rasta WCT               715
     Duck Trousers           655
     Atles Lussekofta        639
2016 Rossi Bermuda Shorts    589
     Game Over T-Shirt       504
     High Heels Shoes        490
     Sheat Shoes             458
     Feiss Fleece Trousers   404
2015 Car Boots               334
     Rossi Bermuda Shorts    279
     High Heels Shoes        248
     Okkaba Skin Jackets     243
     Samba Socker  Socks     237
2014 LA. Shorts              100
     Wimbledon T-Shirt        70
     Mehmet-Tröja             60
     Sheat Shoes              55
     Rasta WCT                45

##### Os 5 produtos mais vendidos no departamento **Clothing** durante todo o periodo analisado

In [13]:
data.query('Departamento == "Clothing"').groupby('Produto').Quantidade.agg(Qtde = 'sum').sort_values('Qtde',  ascending=False).head()

,Qtde
Produto,
Game Over T-Shirt,1583
Rasta WCT,1496
Rossi Bermuda Shorts,1397
Bow tie,1315
Wimbledon T-Shirt,1181


##### Os 5 produtos que mais vendem no departamento **Shoes** durante todo o periodo analisado

In [14]:
data.query('Departamento == "Shoes"').groupby('Produto').Quantidade.agg(Qtde = 'sum').sort_values('Qtde', ascending=False).head()

,Qtde
Produto,
High Heels Shoes,1263
TieBreak Tennis shoes,978
Snake Boots,889
WFS shoes,753
Davenport,746


##### O produto que mais vendeu em cada ano no departamento **Shoes**

In [16]:
data.query('Departamento == "Shoes"').groupby(['Ano', 'Produto']).Quantidade.agg(Qtde = 'sum').sort_values(['Ano','Qtde'], ascending=False).groupby(level=0).head(1)

,,Qtde
Ano,Produto,
2018,Balett Shoes,113
2017,Snake Boots,510
2016,High Heels Shoes,490
2015,High Heels Shoes,248
2014,DSW,40


No departamento **Shoes** o produto destaque corresponde ao **High Heels Shoes** cujo qual foi mais vendido em dois anos: 2015 e 2016. 

##### O produto que mais vendeu em cada ano no departamento **Clothing**

In [17]:
data.query('Departamento == "Clothing"').groupby(['Ano', 'Produto']).Quantidade.agg(Qtde = 'sum').sort_values(['Ano','Qtde'], ascending=False).groupby(level=0).head(1)

,,Qtde
Ano,Produto,
2018,Bow tie,234
2017,Game Over T-Shirt,863
2016,Rossi Bermuda Shorts,589
2015,Car Boots,334
2014,LA. Shorts,100


Não foi possível um produto destaque para o departmento **Clothing** tendo em vista que em todos os anos um produto diferente prevaleceu em 1º lugar de vendas.

##### Quantidade de produtos vendidos por escritório em cada ano 

In [18]:
data.groupby(['Ano', 'EscritorioLocalizacao']).Quantidade.agg(Qtde = 'sum').sort_values(['Ano','Qtde'],  ascending=False).groupby(level=0).head(5)

Qtde
Ano  EscritorioLocalizacao      
2018 Stockholm              1575
     Lund                   1091
     Paris                   559
     Seattle                 531
     Nice                    123
2017 Stockholm              9155
     Lund                   7520
     Seattle                4230
     Paris                  2301
     Nice                   1143
2016 Stockholm              5437
     Lund                   3913
     Seattle                2549
     Paris                  1597
     Nice                   1027
2015 Stockholm              3973
     Seattle                1731
     Lund                   1570
     Nice                    646
     Paris                   550
2014 Stockholm               464
     Lund                    322
     Nice                     50

A partir desses dados podemos visualizar a expansão de dois escritórios em lugares diferentes (Seattle e Paris) de 2014 para 2015. Além disso, visualiza-se que o escritório em Stockholm é o responsável pelo maior número de vendas independentemente do ano analisado.  

##### O produto mais vendido por escritório e a sua respectividade quantidade 

In [19]:
data.groupby(['Ano', 'EscritorioLocalizacao', 'Produto']).Quantidade.agg(Qtde = 'sum').sort_values(['Ano','EscritorioLocalizacao'], ascending=False).groupby(level=[0,1]).head(1)

Qtde
Ano  EscritorioLocalizacao Produto                    
2018 Stockholm             Aino Shoes              104
     Seattle               Aino Shoes                7
     Paris                 Baby Dark Lounge Suit    10
     Nice                  Bow tie                  10
     Lund                  Atles Lussekofta         20
2017 Stockholm             Adihash Running Shoe     71
     Seattle               Adihash Running Shoe     20
     Paris                 Adihash Running Shoe     44
     Nice                  Aino Shoes               42
     Lund                  Adihash Running Shoe    142
2016 Stockholm             Adihash Running Shoe     30
     Seattle               Aino Shoes               10
     Paris                 Adihash Running Shoe     20
     Nice                  Aino Shoes               13
     Lund                  Adihash Running Shoe    136
2015 Stockholm             Adihash Running Shoe    115
     Seattle               Adihash Running Shoe     25
     Paris                 Conserve Shoes           10
     Nice                  Balett Shoes             40
     Lund                  Aino Shoes               28
2014 Stockholm             Aino Shoes               20
     Nice                  Cap                      20
     Lund                  Bike Helmet              24

#### Produtos e os resultados financeiros

##### O produto que gerou o maior faturamento em cada ano, o respectivo valor e a proporção em relação ao faturamento total

In [20]:
#Criando a variável Faturamento
data['Faturamento'] = data["Valor"] * data["Quantidade"]

In [21]:
df_maiores = data.groupby(['Ano','Produto']).Faturamento.agg(Faturamento = 'sum')
df_maiores ['%'] = 100 * df_maiores['Faturamento'] / df_maiores.groupby('Produto')['Faturamento'].transform('sum')
df_maiores.sort_values(['Ano', 'Faturamento'],ascending=False).groupby('Ano').head(1)

,,Faturamento,%
Ano,Produto,,
2018,Minnki Pälsii,2138635.54,12.430174
2017,Minnki Pälsii,8355809.50,48.565622
2016,Minnki Pälsii,6622814.48,38.493111
2015,Okkaba Skin Jackets,1446044.54,66.661924
2014,Okkaba Skin Jackets,73065.68,3.368291


Observamos aqui que apesar do produto **Minnki Pälsii** não aparecer sempre como o produto com a maior quantidade vendida, tem-se o resultado que o mesmo foi o responsável pelo maior faturamento entre 2016 a 2017. Outro destaque é em relação ao produto **Okkaba Skin Jackets** que especialmente em 2015 representou 66,66% do faturamento total. 

##### Conhecendo a margem bruta por produto em cada ano e o seu respectivo valor 

In [41]:
#Variável lucro bruto
data["LucroBruto"] = data["Valor"] - data["Custo"]

#Variável margem de lucro bruto
data['Margem Bruta'] = (data["LucroBruto"]/data["Valor"])*100

##### Descrição da margem bruta dos produtos

In [42]:
data['Margem Bruta'].describe()

count    2172.000000
mean       19.187783
std         6.809344
min         2.424682
25%        13.600000
50%        19.000000
75%        23.304348
max        35.211621
Name: Margem Bruta, dtype: float64

##### Produtos com maior margem bruta por ano 

In [18]:
df = data[["Ano","Produto", "Margem Bruta"]]
df.sort_values(['Margem Bruta', 'Ano'],ascending=False).groupby('Ano').head(1)

,Ano,Produto,Margem Bruta
1017,2017,Rodbye Troje,35.211621
421,2016,Squash Shorts,35.204397
678,2018,Snake Boots,35.200080
794,2015,Basket Vest,33.762641
122,2014,Rasta WCT,30.881007


Aqui visualizamos que os produtos com maiores margens brutas de lucro não são aqueles que tiveram maior parte no faturamento da empresa. 

#### Conclusões

Através dessa análise estatística descritiva, é possível ter alguns insights a respeito do perfil de vendas dos produtos da companhia. Essa análise pode ser útil para que os stakeholders possam trabalhar em estratégias e em tomadas de decisão melhor balizadas no desempenho histórico dos produtos de acordo com o departamento e a localidade.

Por exemplo, uma estratégia que aumente a quantidade de vendas dos produtos que oferecem maior margem bruta tem o potencial de melhorar o resultado anual da companhia, uma vez que esses produtos representam relativamente pouco, quando levamos em consideração os produtos mais vendidos. Em outro exemplo, observamos que o produto Minnki Pälsii representa o maior faturamento da empresa por dois anos, embora não tenha sido o produto mais vendido em nenhum desses anos. Com base nisso, a companhia pode rever a sua política de preços: caso se trate de um produto com demanda mais inelástica, um aumento marginal nos preços desse produto pode levar a uma maior margem (visto que se trata de um produto mais caro, pois gera o maior faturamento sem ser o maior vendedor em quantidade); ou, caso seja um produto com demanda mais elástica, uma queda marginal nos preços pode aumentar a quantidade de vendas, e provocar com isso um aumento ainda maior na receita desse produto.

As vantagens de se extrair informações dos dados são grandes, com inúmeros benefícios. Nesse caso, foi feita uma análise ampla e abrangente, para se ter uma visão geral do perfil de vendas dos produtos da companhia. Em um segundo passo, é possível realizar uma análise mais segmentada, mais minuciosa ou que objetiva estudar mais a fundo algum dos ramos de negócios da companhia. A partir do feedback dos tomadores de decisão a respeito desse relatório, um estudo feito sob medida será capaz de responder perguntas objetivas e melhorar a performance financeira e a capacidade de geração de valor da companhia.
